In [20]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime as dt
from arch import arch_model
from Utils import *

# SPX implied volatility data

In [3]:
spx_vol = pd.read_csv('processed_data/test.csv')

In [4]:
spx_vol.head(10)

,Unnamed: 0,Unnamed: 1,10,30,60,91,122,152,182,273,365,547,730
0,2020-01-02,-90,0.000494,0.000381,0.000469,0.000476,0.000477,0.000485,0.000490,0.000495,0.000502,0.000513,0.000527
1,2020-01-02,-85,0.000431,0.000379,0.000457,0.000473,0.000478,0.000488,0.000494,0.000507,0.000517,0.000534,0.000549
2,2020-01-02,-80,0.000391,0.000384,0.000456,0.000477,0.000487,0.000497,0.000505,0.000522,0.000533,0.000557,0.000577
3,2020-01-02,-75,0.000379,0.000393,0.000463,0.000488,0.000501,0.000512,0.000519,0.000540,0.000553,0.000580,0.000606
4,2020-01-02,-70,0.000381,0.000405,0.000476,0.000503,0.000519,0.000530,0.000537,0.000562,0.000576,0.000604,0.000632
5,2020-01-02,-65,0.000386,0.000419,0.000492,0.000522,0.000539,0.000549,0.000557,0.000587,0.000602,0.000629,0.000657
6,2020-01-02,-60,0.000394,0.000436,0.000510,0.000543,0.000560,0.000571,0.000579,0.000615,0.000631,0.000657,0.000683
7,2020-01-02,-55,0.000404,0.000454,0.000532,0.000566,0.000584,0.000595,0.000604,0.000646,0.000663,0.000687,0.000711
8,2020-01-02,-50,0.000416,0.000475,0.000556,0.000593,0.000612,0.000623,0.000633,0.000679,0.000697,0.000720,0.000741
9,2020-01-02,-45,0.000430,0.000498,0.000583,0.000622,0.000643,0.000655,0.000666,0.000716,0.000734,0.000756,0.000775


# SPX price data

In [5]:
spx = pd.read_csv('data/SPX.csv', skiprows=2)

In [6]:
spx.shape

(9159, 6)

In [16]:
spx['Date'] = pd.to_datetime(spx['Date'], format='%m/%d/%Y')

In [17]:
spx["Return"] = convert_to_daily_returns(spx.Close)

In [18]:
spx.head(10)

,Date,Ticker,Open,High,Low,Close,Return
0,1986-01-02,_SPXD,211.24,211.28,208.93,209.59,NaN
1,1986-01-03,_SPXD,209.60,210.88,209.51,210.88,0.006155
2,1986-01-06,_SPXD,210.89,210.98,209.93,210.65,-0.001091
3,1986-01-07,_SPXD,210.69,213.80,210.65,213.80,0.014954
4,1986-01-08,_SPXD,213.79,214.57,207.49,207.97,-0.027268
5,1986-01-09,_SPXD,207.96,207.97,204.51,206.11,-0.008944
6,1986-01-10,_SPXD,206.17,207.33,205.52,205.96,-0.000728
7,1986-01-13,_SPXD,205.97,206.83,205.52,206.72,0.003690
8,1986-01-14,_SPXD,206.75,207.37,206.06,206.64,-0.000387
9,1986-01-15,_SPXD,206.64,208.27,206.64,208.26,0.007840


# Baseline: Linear Regression

In [19]:
spx.shift(-1).head(10)

,Date,Ticker,Open,High,Low,Close,Return
0,1986-01-03,_SPXD,209.60,210.88,209.51,210.88,0.006155
1,1986-01-06,_SPXD,210.89,210.98,209.93,210.65,-0.001091
2,1986-01-07,_SPXD,210.69,213.80,210.65,213.80,0.014954
3,1986-01-08,_SPXD,213.79,214.57,207.49,207.97,-0.027268
4,1986-01-09,_SPXD,207.96,207.97,204.51,206.11,-0.008944
5,1986-01-10,_SPXD,206.17,207.33,205.52,205.96,-0.000728
6,1986-01-13,_SPXD,205.97,206.83,205.52,206.72,0.003690
7,1986-01-14,_SPXD,206.75,207.37,206.06,206.64,-0.000387
8,1986-01-15,_SPXD,206.64,208.27,206.64,208.26,0.007840
9,1986-01-16,_SPXD,208.31,209.18,207.61,209.17,0.004370


# Baseline: GARCH